In [111]:
import pandas as pd
import numpy as np
gdp_data=pd.read_excel("excel_files//gdp_changeqtr_change.xlsx")

gdp_data.head()

,Quarter,GDP % Change
0,1996 July,0.5
1,1996 Oct,1.6
2,1997 Jan,0.8
3,1997 Apr,0.4
4,1997 July,1.2


In [112]:
def monthly_calculator(prev_gdp,next_gdp,curr_month, start_month):
    change=(next_gdp-prev_gdp)/3
    value=prev_gdp+(curr_month-start_month)*change
    return value
    

In [113]:
def monthly_converter(df):
    i = 0
    new_rows = []
    while i < (len(df) - 1):
        month = str(df['Quarter'][i])[6:10].strip()  # Remove leading/trailing whitespaces
        yr = str(df['Quarter'][i])[1:5]
        
        gdp1 = df['GDP % Change'][i]
        gdp2 = df['GDP % Change'][i+1]
        
        if month == 'Jan':
            new_rows.append(['01/'+ yr , gdp1])
            new_rows.append(['02/'+ yr, monthly_calculator(gdp1, gdp2, 2, 1)])
            new_rows.append(['03/'+ yr, monthly_calculator(gdp1, gdp2, 3, 1)])
        elif month == 'Apr':
            new_rows.append(['04/'+ yr, gdp1])
            new_rows.append(['05/'+ yr, monthly_calculator(gdp1, gdp2, 5, 4)])
            new_rows.append(['06/'+ yr, monthly_calculator(gdp1, gdp2, 6, 4)])
        elif month == 'July':
            new_rows.append(['07/'+ yr, gdp1])
            new_rows.append(['08/'+ yr, monthly_calculator(gdp1, gdp2, 8, 7)])
            new_rows.append(['09/'+ yr, monthly_calculator(gdp1, gdp2, 9, 7)])
        else:
            new_rows.append(['10/'+ yr, gdp1])
            new_rows.append(['11/'+ yr, monthly_calculator(gdp1, gdp2, 11, 10)])
            new_rows.append(['12/' + yr, monthly_calculator(gdp1, gdp2, 12, 10)])
        
        
        
        i += 1
    
    
    new_rows.append([df['Quarter'].iloc[-1], df['GDP % Change'].iloc[-1]])
    
   
    new_df = pd.DataFrame(new_rows, columns=['Quarter', 'GDP % Change'])
    

    
    return new_df


In [114]:
monthly_data=monthly_converter(gdp_data)
print(monthly_data.head(20))

    Quarter  GDP % Change
0   07/1996      0.500000
1   08/1996      0.866667
2   09/1996      1.233333
3   10/1996      1.600000
4   11/1996      1.333333
5   12/1996      1.066667
6   01/1997      0.800000
7   02/1997      0.666667
8   03/1997      0.533333
9   04/1997      0.400000
10  05/1997      0.666667
11  06/1997      0.933333
12  07/1997      1.200000
13  08/1997      1.333333
14  09/1997      1.466667
15  10/1997      1.600000
16  11/1997      1.600000
17  12/1997      1.600000
18  01/1998      1.600000
19  02/1998      1.500000


In [115]:

monthly_data.to_excel('monthly_gdp_data1.xlsx', index=False)


In [122]:
import os
import pandas as pd

# Set the directory where the Excel files are located
directory = "C:\\Users\\91978\\Desktop\\inflation_prediction_project\\excel_files"

# Initialize an empty list to store DataFrames
dfs = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.xlsx'):  # Check if the file is an Excel file
        filepath = os.path.join(directory, filename)
        # Read the Excel file into a DataFrame
        df = pd.read_excel(filepath)
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate the DataFrames along the rows
combined_df = pd.concat(dfs, ignore_index=True)

# Handle missing dates if necessary (e.g., fill or drop)
# combined_df = combined_df.dropna(subset=['Date'])

# Write the combined DataFrame to a new Excel file
output_file="C:\\Users\\91978\\Desktop\\inflation_prediction_project\\excel_files\\combine_dataset.xlsx"

combined_df.to_excel(output_file, index=False,engine='xlsxwriter')


Extrapolation of unemployment data in monthly format using X-12-ARIMA

In [263]:
yoy=pd.read_excel('C:\\Users\\91978\\Desktop\\inflation_prediction_project\\yoy_unemployment_rate.xlsx')
monthly_data=pd.read_excel('C:\\Users\\91978\\Desktop\\inflation_prediction_project\\excel_files\\unemployment_monthly.xlsx')




In [264]:
import pandas as pd



# Convert the 'Date' column to datetime format
monthly_data['Date'] = pd.to_datetime(monthly_data['Date'], format='%m/%Y')

# Sort the dataframe by date
monthly_data.sort_values('Date', inplace=True)

# Display the first few rows of the dataframe
print(monthly_data.head())

# Now you can proceed with the X-13 ARIMA analysis using the 'Date' and 'Unemployment Rate (%)' columns.


        Date  Unemployment Rate (%)
0 2016-04-01                    8.9
1 2016-05-01                    9.7
2 2016-06-01                    8.9
3 2016-07-01                    8.5
4 2016-08-01                    9.6


In [265]:
import os
print(os.environ['X12PATH'])
print(os.environ['X13PATH'])


C:\Users\91978\Desktop\inflation_prediction_project\Time_Sereis_models\x12a.exe
C:\Users\91978\Desktop\inflation_prediction_project\Time_Sereis_models\x13as.exe


In [266]:

import pandas as pd
from statsmodels.tsa.x13 import x13_arima_analysis
from pandas import DatetimeIndex
#  Strip leading/trailing whitespaces from column names
yoy.columns = yoy.columns.str.strip()


# Convert the 'date' column to datetime format
yoy['date'] = pd.to_datetime(yoy['date'])

# Create a new index with monthly frequency
yoy.index = pd.date_range(start=yoy['date'].min(), periods=len(yoy), freq='MS')

# Convert the 'Date' column to datetime format
monthly_data['Date'] = pd.to_datetime(monthly_data['Date'])

# Create a new index with monthly frequency
monthly_data.index = pd.date_range(start=monthly_data['Date'].min(), periods=len(monthly_data), freq='MS')

# Apply X-12-ARIMA to the yearly data
result_yearly = x13_arima_analysis(yoy['Unemployment Rate (%)'], x12path="C:\\Users\\91978\\Desktop\\inflation_prediction_project\\Time_Sereis_models\\x12a.exe")

# Get the seasonal adjusted data for yearly data
seasonal_adjusted_yearly = result_yearly.seasadj

# Apply X-12-ARIMA to the monthly data
result_monthly = x13_arima_analysis(monthly_data['Unemployment Rate (%)'], x12path="C:\\Users\\91978\\Desktop\\inflation_prediction_project\\Time_Sereis_models\\x12a.exe")

# Get the seasonal adjusted data for monthly data
seasonal_adjusted_monthly = result_monthly.seasadj

X13Error: ERROR: Series to be modelled and/or seasonally adjusted must have at
        least 3 complete years of data.
  
  No seasonal adjustment this run

In [267]:
combined_data.head()

,date,Unemployment Rate (%),Annual Change,,Date,Unemployment Rate (%)
0,1991-12-01,6.737,NaN,NaN,NaT,NaN
1,1992-12-01,6.815,0.08,NaN,NaT,NaN
2,1993-12-01,6.798,-0.02,NaN,NaT,NaN
3,1994-12-01,6.830,0.03,NaN,NaT,NaN
4,1995-12-01,7.014,0.18,NaN,NaT,NaN


In [268]:
yoy.head()

,date,Unemployment Rate (%),Annual Change,
1991-12-01,1991-12-01,6.737,NaN,NaN
1992-01-01,1992-12-01,6.815,0.08,NaN
1992-02-01,1993-12-01,6.798,-0.02,NaN
1992-03-01,1994-12-01,6.830,0.03,NaN
1992-04-01,1995-12-01,7.014,0.18,NaN


In [269]:
monthly.head()

,Date,Unemployment Rate (%)
0,2016-04-01,8.9
1,2016-05-01,9.7
2,2016-06-01,8.9
3,2016-07-01,8.5
4,2016-08-01,9.6


In [270]:
print(yoy.columns)


Index(['date', 'Unemployment Rate (%)', 'Annual Change', ''], dtype='object')


In [271]:
yoy['date']

1991-12-01   1991-12-01
1992-01-01   1992-12-01
1992-02-01   1993-12-01
1992-03-01   1994-12-01
1992-04-01   1995-12-01
1992-05-01   1996-12-01
1992-06-01   1997-12-01
1992-07-01   1998-12-01
1992-08-01   1999-12-01
1992-09-01   2000-12-01
1992-10-01   2001-12-01
1992-11-01   2002-12-01
1992-12-01   2003-12-01
1993-01-01   2004-12-01
1993-02-01   2005-12-01
1993-03-01   2006-12-01
1993-04-01   2007-12-01
1993-05-01   2008-12-01
1993-06-01   2009-12-01
1993-07-01   2010-12-01
1993-08-01   2011-12-01
1993-09-01   2012-12-01
1993-10-01   2013-12-01
1993-11-01   2014-12-01
1993-12-01   2015-12-01
1994-01-01   2016-12-01
1994-02-01   2017-12-01
1994-03-01   2018-12-01
1994-04-01   2019-12-01
1994-05-01   2020-12-01
1994-06-01   2021-12-01
1994-07-01   2022-12-01
Freq: MS, Name: date, dtype: datetime64[ns]

In [272]:
monthly_data.columns

Index(['Date', 'Unemployment Rate (%)'], dtype='object')

In [273]:
import pandas as pd
from statsmodels.tsa.x13 import x13_arima_analysis

# Strip leading/trailing whitespaces from column names
yoy.columns = yoy.columns.str.strip()

# Convert the 'date' column to datetime format
yoy['date'] = pd.to_datetime(yoy['date'])

# Create a new index with monthly frequency for yearly data
yoy.index = pd.date_range(start=yoy['date'].min(), periods=len(yoy), freq='MS')

# Convert the 'Date' column to datetime format
monthly_data['Date'] = pd.to_datetime(monthly_data['Date'])

# Create a new index with monthly frequency for monthly data
monthly_data.index = pd.date_range(start=monthly_data['Date'].min(), periods=len(monthly_data), freq='MS')

# Ensure that the data covers at least 3 complete years
if len(yoy) < 3:
    raise ValueError("Yearly data must cover at least 3 complete years")

if len(monthly_data) < 36:
    raise ValueError("Monthly data must cover at least 3 complete years")

# Apply X-12-ARIMA to the yearly data
result_yearly = x13_arima_analysis(yoy['Unemployment Rate (%)'], x12path="C:\\Users\\91978\\Desktop\\inflation_prediction_project\\Time_Sereis_models\\x12a.exe")

# Get the seasonal adjusted data for yearly data
seasonal_adjusted_yearly = result_yearly.seasadj

# Apply X-12-ARIMA to the monthly data
result_monthly = x13_arima_analysis(monthly_data['Unemployment Rate (%)'], x12path="C:\\Users\\91978\\Desktop\\inflation_prediction_project\\Time_Sereis_models\\x12a.exe")

# Get the seasonal adjusted data for monthly data
seasonal_adjusted_monthly = result_monthly.seasadj


X13Error: ERROR: Series to be modelled and/or seasonally adjusted must have at
        least 3 complete years of data.
  
  No seasonal adjustment this run

In [274]:
import pandas as pd
from statsmodels.tsa.x13 import x13_arima_analysis

# Convert the 'date' column to datetime format
yoy['date'] = pd.to_datetime(yoy['date'])

# Ensure the index is set to 'date'
yoy.set_index('date', inplace=True)

# Set the frequency of the index to 'AS' (Year Start)
yoy.index.freq = 'AS'

# Ensure there are at least 3 complete years of data
if len(yoy) < 3:
    raise ValueError("Yearly data must cover at least 3 complete years")

# Apply X-13 ARIMA-SEATS to the yearly data
result_yearly = x13_arima_analysis(yoy['Unemployment Rate (%)'], x12path="C:\\Users\\91978\\Desktop\\inflation_prediction_project\\Time_Sereis_models\\x12a.exe")

# Get the seasonal adjusted data for yearly data
seasonal_adjusted_yearly = result_yearly.seasadj


ValueError: Inferred frequency AS-DEC from passed values does not conform to passed frequency AS-JAN

In [361]:
yoy=pd.read_excel('C:\\Users\\91978\\Desktop\\inflation_prediction_project\\yoy_unemployment_rate.xlsx')
monthly_data=pd.read_excel('C:\\Users\\91978\\Desktop\\inflation_prediction_project\\excel_files\\unemployment_monthly.xlsx')




In [362]:
import pandas as pd



# Convert the 'Date' column to datetime format
monthly_data['Date'] = pd.to_datetime(monthly_data['Date'], format='%m/%Y')

# Sort the dataframe by date
monthly_data.sort_values('Date', inplace=True)

# Display the first few rows of the dataframe
print(monthly_data.head())

# Now you can proceed with the X-13 ARIMA analysis using the 'Date' and 'Unemployment Rate (%)' columns.
print(yoy.columns)

        Date  Unemployment Rate (%)
0 2016-04-01                    8.9
1 2016-05-01                    9.7
2 2016-06-01                    8.9
3 2016-07-01                    8.5
4 2016-08-01                    9.6
Index(['date', ' Unemployment Rate (%)', ' Annual Change', ' '], dtype='object')


In [363]:
outlier_months = [1, 5, 9]

# Extract the 'Date' column from the DataFrame
dates = pd.to_datetime(monthly_data['Date'])

# Initialize the outlier series with zeros
outliers_series = pd.Series(0, index=monthly_data.index)

# Set outliers to 1 for the specified months
outliers_series.loc[dates.dt.month.isin(outlier_months)] = 1

print(outliers_series)
print(yoy.columns)

0     0
1     1
2     0
3     0
4     0
     ..
76    0
77    1
78    0
79    0
80    0
Length: 81, dtype: int64
Index(['date', ' Unemployment Rate (%)', ' Annual Change', ' '], dtype='object')


In [365]:
import pandas as pd
from statsmodels.tsa.x13 import x13_arima_analysis

# Assuming 'yoy' contains yearly data and 'monthly_data' contains monthly data

# Ensure that the column names have no leading/trailing whitespaces
yoy.columns = yoy.columns.str.strip()

# Convert the 'date' column in yearly data to datetime format
yoy.index = pd.to_datetime(yoy.index)

# Filter the monthly data to include only observations from April 2016 to December 2022
# If you want monthly data from 1991, remove the filtering
monthly_data_filtered = monthly_data.loc['1991-01-01':'2022-12-31']

# Apply X-12-ARIMA to the yearly data if it covers at least 3 complete years
if len(yoy.index.year.unique()) >= 3:
    # Resample yearly data to quarterly frequency
    yoy_quarterly = yoy.resample('Q').mean()['Unemployment Rate (%)']
    # Fill missing values using forward fill
    yoy_quarterly = yoy_quarterly.fillna(method='ffill')
    # Check for missing values
    if yoy_quarterly.isnull().sum() > 0:
        print("Warning: Missing values detected in yoy_quarterly data.")
    
    # Specify the series name and other relevant information
    yoy_quarterly.name = 'Unemployment Rate (%)'
    yoy_quarterly.attrs['description'] = 'Quarterly Unemployment Rate'
    # Initialize the outlier series with zeros
    outliers_series = pd.Series(0, index=yoy_quarterly.index)
    # Set outliers to 1 for the specified months
    outlier_months = [1, 5, 9]
    outliers_series.loc[yoy_quarterly.index.month.isin(outlier_months)] = 1
    # Apply outlier adjustment to the series
    yoy_quarterly = yoy_quarterly.copy()
    yoy_quarterly.attrs['outlier'] = outliers_series
    
    # Apply X-12-ARIMA to the yearly data
    result_yearly = x13_arima_analysis(yoy_quarterly, x12path="C:\\Users\\91978\\Desktop\\inflation_prediction_project\\Time_Sereis_models\\x12a.exe")
    seasonal_adjusted_yearly = result_yearly.seasadj
else:
    print("Error: Yearly data must cover at least 3 complete years.")

# Apply X-12-ARIMA to the monthly data
result_monthly = x13_arima_analysis(monthly_data_filtered['Unemployment Rate (%)'], x12path="C:\\Users\\91978\\Desktop\\inflation_prediction_project\\Time_Sereis_models\\x12a.exe")
seasonal_adjusted_monthly = result_monthly.seasadj


Error: Yearly data must cover at least 3 complete years.


TypeError: cannot infer freq from a non-convertible index of dtype int64

In [334]:
yoy.columns

Index(['Unemployment Rate (%)', 'Annual Change', ''], dtype='object')